# Modeling and Evaluation

In [2]:
# Import library
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression, LassoCV, RidgeCV, ElasticNetCV
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import r2_score

### Read cleaned final csv files.

In [71]:
df = pd.read_csv('../datasets/train_cleaned_final.csv')
df.drop('Unnamed: 0', axis = 1, inplace = True)

df_to_pred = pd.read_csv('../datasets/test_cleaned_final.csv')
df_to_pred.drop('Unnamed: 0', axis = 1, inplace = True)

#### Set up the predictors and target

In [4]:
features = df.columns.drop('saleprice')
X = df[features]
y = df['saleprice']

In [5]:
X.head()

,lot_frontage,lot_area,year_built,year_remod/add,mas_vnr_area,bsmtfin_sf_1,bsmtfin_sf_2,bsmt_unf_sf,total_bsmt_sf,1st_flr_sf,...,garage_type_NA,sale_type_COD,sale_type_CWD,sale_type_Con,sale_type_ConLD,sale_type_ConLI,sale_type_ConLw,sale_type_New,sale_type_Oth,sale_type_WD
0,0.0,13517.0,1976.0,2005.0,289.0,533.0,0.0,192.0,725.0,725.0,...,0,0,0,0,0,0,0,0,0,1
1,43.0,11492.0,1996.0,1997.0,132.0,637.0,0.0,276.0,913.0,913.0,...,0,0,0,0,0,0,0,0,0,1
2,68.0,7922.0,1953.0,2007.0,0.0,731.0,0.0,326.0,1057.0,1057.0,...,0,0,0,0,0,0,0,0,0,1
3,73.0,9802.0,2006.0,2007.0,0.0,0.0,0.0,384.0,384.0,744.0,...,0,0,0,0,0,0,0,0,0,1
4,82.0,14235.0,1900.0,1993.0,0.0,0.0,0.0,676.0,676.0,831.0,...,0,0,0,0,0,0,0,0,0,1


### Let's use the numeric features to get some polynomial features

In [6]:
# Use PolynomialFeatures.
poly = PolynomialFeatures(include_bias=False)

# Fit and transform.
X_poly = poly.fit_transform(X)

In [7]:
# Polynomial feature names.
poly_col_names = poly.get_feature_names(features)

In [8]:
# Polynomeal features of numeric valus columns.
X_poly_df = pd.DataFrame(X_poly, columns=poly_col_names)
X_poly_df.head()

,lot_frontage,lot_area,year_built,year_remod/add,mas_vnr_area,bsmtfin_sf_1,bsmtfin_sf_2,bsmt_unf_sf,total_bsmt_sf,1st_flr_sf,...,sale_type_ConLw^2,sale_type_ConLw sale_type_New,sale_type_ConLw sale_type_Oth,sale_type_ConLw sale_type_WD,sale_type_New^2,sale_type_New sale_type_Oth,sale_type_New sale_type_WD,sale_type_Oth^2,sale_type_Oth sale_type_WD,sale_type_WD ^2
0,0.0,13517.0,1976.0,2005.0,289.0,533.0,0.0,192.0,725.0,725.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,43.0,11492.0,1996.0,1997.0,132.0,637.0,0.0,276.0,913.0,913.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,68.0,7922.0,1953.0,2007.0,0.0,731.0,0.0,326.0,1057.0,1057.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,73.0,9802.0,2006.0,2007.0,0.0,0.0,0.0,384.0,384.0,744.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,82.0,14235.0,1900.0,1993.0,0.0,0.0,0.0,676.0,676.0,831.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


### Combine polynomial features and target.

In [9]:
# This is predictor with saleprice. (Combined data)
df_combined = X_poly_df
df_combined['saleprice'] = df['saleprice']
df_combined.head()

,lot_frontage,lot_area,year_built,year_remod/add,mas_vnr_area,bsmtfin_sf_1,bsmtfin_sf_2,bsmt_unf_sf,total_bsmt_sf,1st_flr_sf,...,sale_type_ConLw sale_type_New,sale_type_ConLw sale_type_Oth,sale_type_ConLw sale_type_WD,sale_type_New^2,sale_type_New sale_type_Oth,sale_type_New sale_type_WD,sale_type_Oth^2,sale_type_Oth sale_type_WD,sale_type_WD ^2,saleprice
0,0.0,13517.0,1976.0,2005.0,289.0,533.0,0.0,192.0,725.0,725.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,130500.0
1,43.0,11492.0,1996.0,1997.0,132.0,637.0,0.0,276.0,913.0,913.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,220000.0
2,68.0,7922.0,1953.0,2007.0,0.0,731.0,0.0,326.0,1057.0,1057.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,109000.0
3,73.0,9802.0,2006.0,2007.0,0.0,0.0,0.0,384.0,384.0,744.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,174000.0
4,82.0,14235.0,1900.0,1993.0,0.0,0.0,0.0,676.0,676.0,831.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,138500.0


## Correlation Matrix

#### It would be nice to visualize the heatmap of the correlation matrix, but we will skip it  since it will take too long.

In [10]:
# plt.figure(figsize=(3,700))
# sns.heatmap(df_combined.corr()[['saleprice']].sort_values('saleprice'), annot = True)

In [11]:
# Filter the column and sort by the correlation descending order.
corr = df_combined.corr()[['saleprice']].sort_values(by='saleprice', ascending = False)
# corr

In [12]:
# Let's take top 100 column(features) that has high correlation with salesprice.
# key_features = corr.head(201).index
# key_features = key_features.drop('saleprice')
# len(key_features)

### Train Test Split

In [57]:
X = df_combined.drop('saleprice', axis = 1)
y = df_combined['saleprice']

In [58]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

#### Standardize the train set, fit and transform.

In [59]:
ss = StandardScaler()
ss.fit(X_train)

X_train_sc = ss.transform(X_train)
X_test_sc = ss.transform(X_test)

In [60]:
# Check the train set and test set.
print(X_train.shape)
print(X_test.shape)

(1502, 43364)
(501, 43364)


#### Try three regression model and get the cross val score.

In [29]:
lr = LinearRegression()
lasso = LassoCV()
ridge = RidgeCV()
elasticnet = ElasticNetCV()

In [18]:
cross_val_score(lr, X_train_sc, y_train, cv=3).mean()

0.816631746525614

In [19]:
cross_val_score(elasticnet, X_train_sc, y_train, cv=3).mean()

/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1943: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1943: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1943: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


0.8687482126457393

In [20]:
cross_val_score(ridge, X_train_sc, y_train, cv=3).mean()

0.8257151435256523

In [21]:
cross_val_score(lasso, X_train_sc, y_train, cv=3).mean()

/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1943: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1943: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1943: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


0.8471360204397599

In [63]:
elasticnet.fit(X_train_sc, y_train)

/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1943: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


ElasticNetCV(alphas=None, copy_X=True, cv='warn', eps=0.001,
       fit_intercept=True, l1_ratio=0.5, max_iter=1000, n_alphas=100,
       n_jobs=None, normalize=False, positive=False, precompute='auto',
       random_state=None, selection='cyclic', tol=0.0001, verbose=0)

In [23]:
elasticnet.score(X_test_sc, y_test)

0.9106897021610761

In [24]:
elasticnet.score(X_train_sc, y_train)

0.9224361460546557

> Since elasticn net regression model performs the best, we will use it to predict the sale price

### Test holdout set

In [49]:
X_holdout = df_to_pred[features]

#### Apply the same polynomial features to the holdout set.

In [52]:
X_poly = poly.fit_transform(X_holdout)
poly_col_names = poly.get_feature_names(features)

X_poly_holdout = pd.DataFrame(X_poly, columns=poly_col_names)
X_poly_holdout.head()

,lot_frontage,lot_area,year_built,year_remod/add,mas_vnr_area,bsmtfin_sf_1,bsmtfin_sf_2,bsmt_unf_sf,total_bsmt_sf,1st_flr_sf,...,sale_type_ConLw^2,sale_type_ConLw sale_type_New,sale_type_ConLw sale_type_Oth,sale_type_ConLw sale_type_WD,sale_type_New^2,sale_type_New sale_type_Oth,sale_type_New sale_type_WD,sale_type_Oth^2,sale_type_Oth sale_type_WD,sale_type_WD ^2
0,69.0,9142.0,1910.0,1950.0,0.0,0.0,0.0,1020.0,1020.0,908.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.0,9662.0,1977.0,1977.0,0.0,0.0,0.0,1967.0,1967.0,1967.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,58.0,17104.0,2006.0,2006.0,0.0,554.0,0.0,100.0,654.0,664.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,60.0,8520.0,1923.0,2006.0,0.0,0.0,0.0,968.0,968.0,968.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.0,9500.0,1963.0,1963.0,247.0,609.0,0.0,785.0,1394.0,1394.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


#### Standard scaler fit

In [61]:
X_poly_holdout_sc = ss.fit_transform(X_poly_holdout)
X_poly_holdout_sc

array([[ 0.35095192, -0.11939513, -1.99213808, ..., -0.05852057,
         0.        ,  0.40526338],
       [-1.65958787, -0.06761065,  0.21280822, ..., -0.05852057,
         0.        ,  0.40526338],
       [ 0.03043108,  0.67350495,  1.16718797, ..., -0.05852057,
         0.        , -2.46753109],
       ...,
       [-0.05698369, -0.20822544, -0.08337859, ..., -0.05852057,
         0.        ,  0.40526338],
       [ 0.0887076 , -0.13353628,  0.01535035, ..., -0.05852057,
         0.        ,  0.40526338],
       [ 0.38009017, -0.19328761, -0.51120399, ..., -0.05852057,
         0.        ,  0.40526338]])

#### Lasso prediction

In [65]:
#elasticnet.transform(X_poly_holdout_sc)
X_holdout['pred_sale_price'] = elasticnet.predict(X_poly_holdout_sc)

#### Drop all features except the predicted sale price.

In [66]:
X_holdout = X_holdout.drop(features, axis=1)

In [67]:
X_holdout.columns = ['SalePrice']

#### Complete the dataframe for Kaggle submission.

In [75]:
df_to_pred_ref = pd.read_csv('../datasets/test.csv')
X_holdout['Id'] = df_to_pred_ref['Id']
X_holdout.set_index('Id', inplace=True)

#### Save it to the csv file.

In [ ]:
X_holdout.to_csv('../my_submission')